In [1]:
from __future__ import print_function
from matplotlib import pyplot as plt
# get_ipython().magic(u'matplotlib auto')
import tensorflow as tf
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
# import torch
from tensorflow import keras
import statistics

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Activation
from tensorflow import reshape
from keras.utils import np_utils
from tensorflow.keras.layers import Conv1D,Conv2D, MaxPooling1D,AveragePooling1D
from tensorflow.keras.layers import SeparableConv1D, Bidirectional
from tensorflow.keras.layers import LocallyConnected2D
from tensorflow.keras.layers import ZeroPadding2D,ZeroPadding1D, MaxPooling2D, Bidirectional
from tensorflow.keras.regularizers import l2,l1
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import CSVLogger,LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

In [3]:
from scipy.io import loadmat
import numpy as np
import pandas as pd
import seaborn as sns
from numpy import mean
from numpy import std
# import coremltools
# from torch import nn, optim
# import torch.nn.functional as F
import scipy.io
from scipy import stats
#from IPython.display import display, HTML
from tensorflow.keras import optimizers
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import TimeDistributed
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing

In [4]:
# The number of steps within one time segment
TIME_PERIODS = 500
# The steps to take from one segment to the next; if this value is equal to
# TIME_PERIODS, then there is no overlap between the segments
STEP_DISTANCE = 500
N_FEATURES = 10

In [5]:
column_names = ['C'+str(j) for j in range(1, N_FEATURES+1)]
lst = ['Class_label']
column_names = column_names+lst
print(column_names)

['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'Class_label']


In [6]:
def read_data_Train(file_path_Train):
    df_Train = pd.read_csv(file_path_Train,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Train['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Train['Class_label'] = df_Train['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Train.dropna(axis=0, how='any', inplace=True)
    return df_Train
def convert_to_float(x):
    try:
        return np.float(x)
    except:
        return np.nan

In [7]:
def show_basic_dataframe_info(dataframe):
    # Shape and how many rows and columns
    print('Number of columns in the dataframe: %i' % (dataframe.shape[1]))
    print('Number of rows in the dataframe: %i\n' % (dataframe.shape[0]))

In [8]:
def read_data_Test(file_path_Test):
    df_Test = pd.read_csv(file_path_Test,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Test['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Test['Class_label'] = df_Test['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Test.dropna(axis=0, how='any', inplace=True)
    return df_Test

In [9]:
 pd.options.display.float_format = "{:,.5f}".format

In [10]:
# 15 function to segment data into trial lengths (trial length =513 samples in this dataset)
def create_segments_and_labels(df, time_steps,step,n_features, label_name):
    segments = []
    labels = []
    for i in range(0, len(df) - time_steps, step):
      for j in range(1, n_features+1):
        L = ('C'+str(j)) 
        segments.append(df[str(L)].values[i: i + time_steps])
      label = stats.mode(df[label_name][i: i + time_steps])[0][0]
      labels.append(label)
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, time_steps, n_features)
    labels = np.asarray(labels)
    return reshaped_segments, labels

In [11]:
verbose, epochs, batch_size = 0, 120, 64

In [12]:
def step_decay(epoch):
   initial_lrate = 1e-3
   drop = 0.1
   epochs_drop = 40.0
   lrate = initial_lrate * tf.math.pow(drop,  
           tf.math.floor((1+epoch)/epochs_drop))
   return lrate

In [20]:
lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)
test_acc=[]

In [21]:
import os
path='/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_naveen/sub_wise_process_TT/'

In [22]:
for i in range(1,28):
    file_path_train=path+'S'+str(i)+'_'+'tr'+'.'+'csv'
    print(file_path_train)
    df_Train=read_data_Train(file_path_train)
    show_basic_dataframe_info(df_Train)
    df_Train.head(5)
    file_path_test=path+'S'+str(i)+'_'+'tt'+'.'+'csv'
    print(file_path_test)
    df_Test=read_data_Test(file_path_test)
    show_basic_dataframe_info(df_Test)
    df_Test.head(5)
    LABEL = 'ActivityEncoded'
    # Transform the labels from String to Integer via LabelEncoder
    le = preprocessing.LabelEncoder()
    # Add a new column to the existing DataFrame with the encoded values
    df_Train[LABEL] = le.fit_transform(df_Train['Class_label'].values.ravel())
    # df_Valid[LABEL] = le.fit_transform(df_Valid['Class_label'].values.ravel())
    df_Test[LABEL] = le.fit_transform(df_Test['Class_label'].values.ravel())
    x_train, y_train = create_segments_and_labels(df_Train,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
    print('x_train shape: ', x_train.shape)
    # print(x_train)
    print(x_train.shape[0], 'training samples')
    print('y_train shape: ', y_train.shape)
    # Set input & output dimensions
    num_time_periods, num_sensors = x_train.shape[1], x_train.shape[2]
    print('num_time_periods',num_time_periods)
    print('num_sensors',num_sensors)
    num_classes = le.classes_.size
    print('class_list',list(le.classes_))
    # input_shape = (num_time_periods,num_sensors)
    # print(input_shape)
    input_shape = (num_time_periods,num_sensors)
    #x_train = x_train.reshape(x_train.shape[0], input_shape)
    print('x_train shape:', x_train[0].shape)
    print('input_shape:', input_shape)
    x_train = x_train.astype('float32')
    # x_train = [torch.tensor(arr, dtype=torch.float32) for arr in x_train]
    # y_train = y_train.astype('float32')
    # print(y_train)
    y_train_hot = np_utils.to_categorical(y_train, num_classes)
    print(y_train_hot)
    # y_train_hot= [torch.tensor(arr, dtype=torch.uint8) for arr in y_train_hot]
    print('New y_train shape: ', y_train_hot.shape)
    x_test, y_test = create_segments_and_labels(df_Test,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
    print('x_test shape: ', x_test.shape)
    # print(x_train)
    print(x_test.shape[0], 'testing samples')
    print('y_test shape: ', y_test.shape)
    # Set input_shape / reshape for Keras
    #x_test = x_test.reshape(x_test.shape[0], input_shape)
    x_test = x_test.astype('float32')
    y_test = y_test.astype('float32')
    y_test_hot = np_utils.to_categorical(y_test, num_classes)
    # n_steps, n_length = 20, 25
    # n_steps, n_length= 10, 50
    # n_steps, n_length= 16, 32
    n_steps, n_length = 25, 20
    n_depth=10
    x_train = x_train.reshape(x_train.shape[0], n_steps, n_length,n_depth)
    print('x_train shape: ', x_train.shape)
    # x_valid = x_valid.reshape(x_valid.shape[0], n_steps, n_length, n_depth)
    # print('x_valid shape: ', x_valid.shape)
    x_test = x_test.reshape(x_test.shape[0], n_steps, n_length,n_depth)
    print('x_test shape: ', x_test.shape)
    n_outputs = y_train_hot.shape[1]
    print('n_outputs',n_outputs)
    # chen architecture
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=32, kernel_size=2, padding='same', kernel_initializer="he_normal",strides=1,kernel_regularizer=l1(1e-04)), \
                               input_shape=(n_steps,n_length,n_depth)))
    model.add(TimeDistributed(BatchNormalization(epsilon=1e-06, momentum=0.95, weights=None)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=4,strides=2)))
    model.add(TimeDistributed(Activation('tanh')))
    model.add(TimeDistributed(Conv1D(filters=16, kernel_size=2,padding="same",kernel_initializer="he_normal",strides=1,kernel_regularizer=l1(1e-04))))
    model.add(TimeDistributed(BatchNormalization(epsilon=1e-06, momentum=0.95, weights=None)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=4,strides=2)))
    model.add(TimeDistributed(Activation('tanh')))
    model.add(TimeDistributed(Dropout(0.2093)))
    model.add(TimeDistributed(Activation('relu')))
    model.add(TimeDistributed(Flatten()))
    # model.add(Flatten())
    model.add(LSTM(64,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(32,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='tanh'))
    model.add(BatchNormalization(epsilon=1e-05, momentum=0.9, weights=None))
    model.add(Dense(n_outputs, activation='softmax'))
    adam=optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, amsgrad=False)
#     sgd=optimizers.SGD(learning_rate=2.17e-3, momentum=0.9, nesterov=False, name='SGD')
    checkpoint_filepath = '/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/checkpoint.hdf5'
    # model.load_weights(checkpoint_filepath) 
    checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath,verbose=1, monitor='val_accuracy',save_weights_only=True,save_best_only=True)
    early = EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
    model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
    print(model.summary())
#     tf.keras.utils.plot_model(model, to_file='/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/Model1.png',show_shapes=True,show_layer_names=True,dpi=96)
    csv_logger = CSVLogger('/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/LSTM_nina_20X100.csv', append=True, separator=';')
    history = model.fit(x_train, y_train_hot, epochs=epochs, batch_size=batch_size, callbacks=[csv_logger,checkpoint_callback,lrate,early],validation_data=(x_test, y_test_hot), verbose=1)
    best_index = history.history['val_accuracy'].index(max(history.history['val_accuracy']))
    print('epoch_number',best_index+1)
    print('train accuracy and validation accuracy', history.history['accuracy'][best_index], history.history['val_accuracy'][best_index])
    model.load_weights(checkpoint_filepath) 
    _, testaccuracy = model.evaluate(x_test, y_test_hot, batch_size=batch_size, verbose=1)
    print('test_accuracy',testaccuracy)
    test_acc.append(testaccuracy)
    print(test_acc)
    print('test_mean for %d subjects:',i)
#     print(test_acc)
    test_mean=statistics.mean(test_acc)
    print(test_mean)
    print('test standard deviation:')
   

/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_naveen/sub_wise_process_TT/S1_tr.csv
Number of columns in the dataframe: 11
Number of rows in the dataframe: 182000

/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_naveen/sub_wise_process_TT/S1_tt.csv
Number of columns in the dataframe: 11
Number of rows in the dataframe: 78000

x_train shape:  (363, 500, 10)
363 training samples
y_train shape:  (363,)
num_time_periods 500
num_sensors 10
class_list [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0]
x_train shape: (500, 10)
input_shape: (500, 10)
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
New y_train shape:  (363, 52)
x

Epoch 12/120
5/6 [========================>.....] - ETA: 0s - loss: 1.4566 - accuracy: 0.6313
Epoch 00012: val_accuracy improved from 0.41290 to 0.57419, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/checkpoint.hdf5
6/6 [==============================] - 0s 29ms/step - loss: 1.4707 - accuracy: 0.6171 - val_loss: 1.8349 - val_accuracy: 0.5742 - lr: 0.0010
Epoch 13/120
5/6 [========================>.....] - ETA: 0s - loss: 1.3255 - accuracy: 0.6719
Epoch 00013: val_accuracy improved from 0.57419 to 0.60645, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/checkpoint.hdf5
6/6 [==============================] - 0s 31ms/step - loss: 1.3215 - accuracy: 0.6722 - val_loss: 1.7245 - val_accuracy: 0.6065 - lr: 0.0010
Epoch 14/120
5/6 [========================>.....] - ETA: 0s - loss: 1.2082 - accuracy: 0.7312
Epoch 00014: val_accuracy improved from 0.60645 to 0.62581, saving model to /media/naveen/nav/mat_co

Epoch 37/120
5/6 [========================>.....] - ETA: 0s - loss: 0.2874 - accuracy: 0.9563
Epoch 00037: val_accuracy did not improve from 0.85161
6/6 [==============================] - 0s 19ms/step - loss: 0.2878 - accuracy: 0.9504 - val_loss: 0.6032 - val_accuracy: 0.8129 - lr: 0.0010
Epoch 38/120
5/6 [========================>.....] - ETA: 0s - loss: 0.2894 - accuracy: 0.9469
Epoch 00038: val_accuracy did not improve from 0.85161
6/6 [==============================] - 0s 20ms/step - loss: 0.2858 - accuracy: 0.9504 - val_loss: 0.5947 - val_accuracy: 0.8194 - lr: 0.0010
Epoch 39/120
6/6 [==============================] - ETA: 0s - loss: 0.2690 - accuracy: 0.9532
Epoch 00039: val_accuracy did not improve from 0.85161
6/6 [==============================] - 0s 21ms/step - loss: 0.2690 - accuracy: 0.9532 - val_loss: 0.5804 - val_accuracy: 0.8387 - lr: 0.0010
Epoch 40/120
5/6 [========================>.....] - ETA: 0s - loss: 0.2401 - accuracy: 0.9563
Epoch 00040: val_accuracy did not im

Epoch 65/120
5/6 [========================>.....] - ETA: 0s - loss: 0.2410 - accuracy: 0.9688
Epoch 00065: val_accuracy did not improve from 0.85806
6/6 [==============================] - 0s 21ms/step - loss: 0.2300 - accuracy: 0.9725 - val_loss: 0.5479 - val_accuracy: 0.8387 - lr: 1.0000e-04
Epoch 66/120
5/6 [========================>.....] - ETA: 0s - loss: 0.2200 - accuracy: 0.9656
Epoch 00066: val_accuracy did not improve from 0.85806
6/6 [==============================] - 0s 20ms/step - loss: 0.2234 - accuracy: 0.9614 - val_loss: 0.5469 - val_accuracy: 0.8387 - lr: 1.0000e-04
Epoch 67/120
6/6 [==============================] - ETA: 0s - loss: 0.2300 - accuracy: 0.9669
Epoch 00067: val_accuracy did not improve from 0.85806
6/6 [==============================] - 0s 19ms/step - loss: 0.2300 - accuracy: 0.9669 - val_loss: 0.5486 - val_accuracy: 0.8516 - lr: 1.0000e-04
Epoch 68/120
6/6 [==============================] - ETA: 0s - loss: 0.2310 - accuracy: 0.9642
Epoch 00068: val_accurac

Epoch 93/120
5/6 [========================>.....] - ETA: 0s - loss: 0.2136 - accuracy: 0.9750
Epoch 00093: val_accuracy did not improve from 0.86452
6/6 [==============================] - 0s 22ms/step - loss: 0.2075 - accuracy: 0.9780 - val_loss: 0.5367 - val_accuracy: 0.8452 - lr: 1.0000e-05
Epoch 94/120
5/6 [========================>.....] - ETA: 0s - loss: 0.1980 - accuracy: 0.9781
Epoch 00094: val_accuracy did not improve from 0.86452
6/6 [==============================] - 0s 22ms/step - loss: 0.1987 - accuracy: 0.9780 - val_loss: 0.5356 - val_accuracy: 0.8516 - lr: 1.0000e-05
Epoch 95/120
5/6 [========================>.....] - ETA: 0s - loss: 0.1945 - accuracy: 0.9781
Epoch 00095: val_accuracy did not improve from 0.86452
6/6 [==============================] - 0s 19ms/step - loss: 0.1963 - accuracy: 0.9752 - val_loss: 0.5361 - val_accuracy: 0.8452 - lr: 1.0000e-05
Epoch 96/120
5/6 [========================>.....] - ETA: 0s - loss: 0.2002 - accuracy: 0.9781
Epoch 00096: val_accurac

...

5/6 [========================>.....] - ETA: 0s - loss: 4.4817 - accuracy: 0.0437
Epoch 00001: val_accuracy improved from -inf to 0.03226, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/checkpoint.hdf5
6/6 [==============================] - 1s 135ms/step - loss: 4.4571 - accuracy: 0.0413 - val_loss: 3.9458 - val_accuracy: 0.0323 - lr: 0.0010
Epoch 2/120
4/6 [===================>..........] - ETA: 0s - loss: 3.6635 - accuracy: 0.1016
Epoch 00002: val_accuracy did not improve from 0.03226
6/6 [==============================] - 0s 35ms/step - loss: 3.6328 - accuracy: 0.1047 - val_loss: 3.8377 - val_accuracy: 0.0258 - lr: 0.0010
Epoch 3/120
5/6 [========================>.....] - ETA: 0s - loss: 3.2300 - accuracy: 0.1219
Epoch 00003: val_accuracy improved from 0.03226 to 0.07097, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/checkpoint.hdf5
6/6 [==============================] - 0s 33ms/step - loss: 3.

6/6 [==============================] - 0s 33ms/step - loss: 0.5691 - accuracy: 0.8926 - val_loss: 0.5056 - val_accuracy: 0.9032 - lr: 0.0010
Epoch 24/120
4/6 [===================>..........] - ETA: 0s - loss: 0.5434 - accuracy: 0.8906
Epoch 00024: val_accuracy improved from 0.90323 to 0.94839, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/checkpoint.hdf5
6/6 [==============================] - 0s 40ms/step - loss: 0.5310 - accuracy: 0.9036 - val_loss: 0.4635 - val_accuracy: 0.9484 - lr: 0.0010
Epoch 25/120
4/6 [===================>..........] - ETA: 0s - loss: 0.4885 - accuracy: 0.8945
Epoch 00025: val_accuracy did not improve from 0.94839
6/6 [==============================] - 0s 26ms/step - loss: 0.4598 - accuracy: 0.9118 - val_loss: 0.4464 - val_accuracy: 0.9355 - lr: 0.0010
Epoch 26/120
5/6 [========================>.....] - ETA: 0s - loss: 0.4116 - accuracy: 0.9531
Epoch 00026: val_accuracy improved from 0.94839 to 0.95484, saving model

Epoch 51/120
6/6 [==============================] - ETA: 0s - loss: 0.2092 - accuracy: 0.9890
Epoch 00051: val_accuracy did not improve from 0.96129
6/6 [==============================] - 0s 19ms/step - loss: 0.2092 - accuracy: 0.9890 - val_loss: 0.2698 - val_accuracy: 0.9484 - lr: 1.0000e-04
Epoch 52/120
6/6 [==============================] - ETA: 0s - loss: 0.1844 - accuracy: 0.9917
Epoch 00052: val_accuracy did not improve from 0.96129
6/6 [==============================] - 0s 19ms/step - loss: 0.1844 - accuracy: 0.9917 - val_loss: 0.2721 - val_accuracy: 0.9484 - lr: 1.0000e-04
Epoch 53/120
5/6 [========================>.....] - ETA: 0s - loss: 0.1844 - accuracy: 0.9812
Epoch 00053: val_accuracy did not improve from 0.96129
6/6 [==============================] - 0s 19ms/step - loss: 0.1861 - accuracy: 0.9835 - val_loss: 0.2698 - val_accuracy: 0.9484 - lr: 1.0000e-04
Epoch 54/120
6/6 [==============================] - ETA: 0s - loss: 0.1840 - accuracy: 0.9917
Epoch 00054: val_accurac

Epoch 79/120
5/6 [========================>.....] - ETA: 0s - loss: 0.1846 - accuracy: 0.9844
Epoch 00079: val_accuracy did not improve from 0.96129
6/6 [==============================] - 0s 20ms/step - loss: 0.1797 - accuracy: 0.9862 - val_loss: 0.2517 - val_accuracy: 0.9548 - lr: 1.0000e-04
Epoch 80/120
6/6 [==============================] - ETA: 0s - loss: 0.1594 - accuracy: 1.0000
Epoch 00080: val_accuracy did not improve from 0.96129
6/6 [==============================] - 0s 18ms/step - loss: 0.1594 - accuracy: 1.0000 - val_loss: 0.2554 - val_accuracy: 0.9613 - lr: 1.0000e-05
Epoch 81/120
5/6 [========================>.....] - ETA: 0s - loss: 0.1504 - accuracy: 0.9906
Epoch 00081: val_accuracy did not improve from 0.96129
6/6 [==============================] - 0s 21ms/step - loss: 0.1533 - accuracy: 0.9890 - val_loss: 0.2521 - val_accuracy: 0.9613 - lr: 1.0000e-05
Epoch 82/120
5/6 [========================>.....] - ETA: 0s - loss: 0.1830 - accuracy: 0.9750
Epoch 00082: val_accurac

Epoch 107/120
5/6 [========================>.....] - ETA: 0s - loss: 0.1618 - accuracy: 0.9875
Epoch 00107: val_accuracy did not improve from 0.96129
6/6 [==============================] - 0s 26ms/step - loss: 0.1638 - accuracy: 0.9835 - val_loss: 0.2513 - val_accuracy: 0.9613 - lr: 1.0000e-05
Epoch 108/120
4/6 [===================>..........] - ETA: 0s - loss: 0.1788 - accuracy: 0.9883
Epoch 00108: val_accuracy did not improve from 0.96129
6/6 [==============================] - 0s 27ms/step - loss: 0.1827 - accuracy: 0.9862 - val_loss: 0.2533 - val_accuracy: 0.9613 - lr: 1.0000e-05
Epoch 109/120
5/6 [========================>.....] - ETA: 0s - loss: 0.1591 - accuracy: 0.9969
Epoch 00109: val_accuracy did not improve from 0.96129
6/6 [==============================] - 0s 21ms/step - loss: 0.1620 - accuracy: 0.9972 - val_loss: 0.2524 - val_accuracy: 0.9613 - lr: 1.0000e-05
Epoch 110/120
5/6 [========================>.....] - ETA: 0s - loss: 0.1512 - accuracy: 0.9906
Epoch 00110: val_acc

In [23]:
statistics.mean(test_acc)
test_std=statistics.stdev(test_acc)
print(test_std)

0.03398379732215286
